# Example of usage

In [ ]:
from stinetwork.test_networks.smallNetwork import initialize_test_network
from stinetwork.visualization.plotting import plot_topology
from stinetwork.loadflow.ac import DistLoadFlow
from stinetwork.visualization.printing import dispVolt, dispFlow, ForwardSearch, BackwardSearch
from load_and_gen_data import WeatherGen,LoadGen,windGen,PVgeneration

## Initialize

Initialize test network:

In [ ]:
ps = initialize_test_network()

Fetch bus-objects:

In [ ]:
B0 = ps.get_comp("B0")
B1 = ps.get_comp("B1")
B2 = ps.get_comp("B2")
B3 = ps.get_comp("B3")
B4 = ps.get_comp("B4")
B5 = ps.get_comp("B5")
M1 = ps.get_comp("M1")
M2 = ps.get_comp("M2")
M3 = ps.get_comp("M3")

Fetch line-objects:

In [ ]:
L1 = ps.get_comp("L1")
L2 = ps.get_comp("L2")
L3 = ps.get_comp("L3")
L4 = ps.get_comp("L4")
L5 = ps.get_comp("L5")
L6 = ps.get_comp("L6")
L7 = ps.get_comp("L7")
ML1 = ps.get_comp("ML1")
ML2 = ps.get_comp("ML2")

Fetch battery objects:

In [ ]:
Bat1 = M1.get_battery()
Bat1.print_status()

Fetch disconnector objects:

In [ ]:
L1a = ps.get_comp("L1a")
L1b = ps.get_comp("L1b")
L1c = ps.get_comp("L1c")
L2a = ps.get_comp("L2a")
L3a = ps.get_comp("L3a")
L3b = ps.get_comp("L3b")
L4a = ps.get_comp("L4a")
L5a = ps.get_comp("L5a")
L6a = ps.get_comp("L6a")
L7a = ps.get_comp("L7a")
L7b = ps.get_comp("L7b")

Get load profiles:

In [ ]:
temp_profiles, wind_profiles, solar_profiles = WeatherGen()

wind = windGen(wind_profiles)
PV = PVgeneration(temp_profiles, solar_profiles)

load_house, load_farm, load_industry2, load_trade, load_office = LoadGen(temp_profiles)

In [ ]:
import numpy as np
print(np.random.choice([True,False],p=[0.2,0.8]))

## Monte Carlo simulation


In [ ]:
N = 1 # Size of Monte Carlo simulation

for i in range(N):
    for day in range(365):
        for hour in range(24):
            ## Set load
            B1.set_load(pload=load_house[day,hour]*10,qload=0.0)
            B2.set_load(pload=load_house[day,hour]*10,qload=0.0)
            B3.set_load(pload=load_house[day,hour]*10,qload=0.0)
            B4.set_load(pload=load_house[day,hour]*10,qload=0.0)
            B5.set_load(pload=load_house[day,hour]*10,qload=0.0)
            M1.set_load(pload=load_house[day,hour]*10,qload=0.0)
            M2.set_load(pload=load_house[day,hour]*10,qload=0.0)
            M3.set_load(pload=load_house[day,hour]*10,qload=0.0)
            
            ## Set fail status
            for comp in ps.get_comp_list():
                comp.update_fail_status()
                
            ## Set backup lines fail status
            for comp in ps.all_lines:
                comp.update_backup_fail_status()
                
            ps.find_sub_systems()
            ps.update_sub_system_slack()
            
            ## Load flow
            for sub_system in ps.sub_systems:
                buses = DistLoadFlow(sub_system["buses"],sub_system["lines"])
            
            ## Print status
            ps.print_status()
            